# Neural Network Exercise 5 (Elman & Jordan Networks)

Amirkabir University of Technology

Dr. Safabakhsh

By Gholamreza Dar 400131018

Spring 2022

## Imports

In [2]:
import os
import datetime

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Functions

### Data loading

In [3]:
def convert_label_to_binary(label):
    if label == 'normal' or label == 'ok':
        return 1
    else:
        return 0

In [4]:
def extract_data(file_path):
    '''Parse the data from 'file_path' and store the number lines in the segment list and the label lines in the labels list'''
    segments = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f.readlines():
            # Number line
            if line.startswith("\t"):
                segments.append(list(map(int, line.split())))
            # Whitespace
            elif line.startswith("\n"):
                pass
            # Label line
            else:
                labels.append(convert_label_to_binary(line[:-1]))

    return segments, labels


In [5]:
def load_data():
    '''Load the data from all 5 files and return a (463, 15, 6) numpy array and labels as a (463,) numpy array'''

    base_dir = "data"
    file_names = ["lp1.data", "lp2.data", "lp3.data", "lp4.data", "lp5.data"]
    
    X = []
    y = []

    # Loop through the files and extract the data and concatenate them
    for file_name in file_names:
        segments, labels = extract_data(os.path.join(base_dir, file_name))
        X += segments
        y += labels
    
    # Convert lists to numpy array and reshape them
    X = np.array(X)
    X = X.reshape(-1, 15, 6).astype(np.float32)
    y = np.array(y).astype(np.float32)
    
    return X, y

### Elman network

In [6]:
class ElmanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(ElmanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            state_t = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))

        y = tf.keras.activations.sigmoid(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))
        return y

    # def evaluate(self, x, y, **kwargs):
    #     y_pred = self(x)
    #     return tf.keras.metrics.binary_accuracy(y, y_pred)

### Jordan Network

In [7]:
class JordanModel(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, feature_size, n_classes):
        super(JordanModel, self).__init__()
        self.hidden_units = hidden_units
        self.feature_size = feature_size
        self.input_dim = input_dim
        self.n_classes = n_classes

        self.W = self.add_weight(shape=(self.hidden_units, self.feature_size), initializer='random_normal', trainable=True)
        self.U = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y_intermediate = self.add_weight(shape=(self.hidden_units, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y_intermediate = self.add_weight(shape=(self.hidden_units,), initializer='random_normal', trainable=True)

        self.W_y = self.add_weight(shape=(self.n_classes, self.hidden_units), initializer='random_normal', trainable=True)
        self.b_y = self.add_weight(shape=(self.n_classes,), initializer='random_normal', trainable=True)

    def call(self, x):
        state_t = tf.zeros(self.hidden_units)
        for i in range(self.input_dim):
            hidden = tf.keras.activations.tanh(tf.matmul(self.W, tf.reshape(x[0][i], (self.feature_size, 1))) + tf.matmul(self.U, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b, (self.hidden_units, 1)))
            y_intermediate = tf.keras.activations.tanh(tf.matmul(self.W_y_intermediate, tf.reshape(hidden, (self.hidden_units,1))) + tf.reshape(self.b_y_intermediate, (self.hidden_units,1)))
            state_t = y_intermediate
        y = tf.keras.activations.sigmoid(tf.matmul(self.W_y, tf.reshape(state_t, (self.hidden_units,1))) + tf.reshape(self.b_y, (self.n_classes,1)))

        return y
    
    # def evaluate(self, x, y):
    #     y_pred = self(x)
    #     return tf.keras.metrics.binary_accuracy(y, y_pred)

### Experiment functions

In [8]:
def train_elman(hidden_count, epochs=30, n_classes=1, lr=1e-3):
    elman_nn = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    elman_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    elman_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    test_score = evaluate_model(elman_nn, X_test, y_test)

    return elman_nn, test_score

In [9]:
def train_jordan(hidden_count, epochs=30, n_classes=1, lr=1e-3):
    jordan_nn = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=n_classes)

    jordan_nn.compile(optimizer=tf.keras.optimizers.Adam(lr),
                    loss='binary_crossentropy',
                    metrics=['acc'])
    jordan_nn.fit(
        X_train,
        y_train,
        batch_size=1,
        epochs=epochs,
        validation_data=(X_validation, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    
    test_score = evaluate_model(jordan_nn, X_test, y_test)

    return jordan_nn, test_score

In [10]:
import sklearn.metrics as metrics

def evaluate_model(model, X, y):
    y_preds = []
    for data in X:
        y_pred = model(np.array([data]))
        y_pred = 1 if np.array(y_pred)[0] > 0.5 else 0
        y_preds.append(y_pred)
    
    return metrics.accuracy_score(y, y_preds)

### Ensemble Functions

In [11]:
def create_elman_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Elman submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        elman_submodel = ElmanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        elman_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=elman_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [12]:
def create_jordan_submodels(hidden_count_list=[10, 25, 50]):
    '''Create a list of Jordan submodels'''
    
    submodels = []
    for hidden_count in hidden_count_list:
        # Create a submodel
        jordan_submodel = JordanModel(input_dim=15, hidden_units=hidden_count, feature_size=6, n_classes=2)
        jordan_submodel.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['acc'])
        
        # Input layer is needed for the ensemble
        input_layer = tf.keras.Input(shape=(15,6,))
        submodel = tf.keras.Model(inputs=input_layer, outputs=jordan_submodel(input_layer))

        submodels.append(submodel)

    return submodels

In [30]:
def create_ensemble_model(sub_models, active_index=None):
    '''This function takes a list of sub-models and returns an ensemble of them.
    Works with elman and jordan sub-models(and any combination of them).'''

    # We have to rename the layers to avoid duplicate layer names in the ensemble model
    for i, sub_model in enumerate(sub_models):
        for layer in sub_model.layers:
            layer._name = f'ensemble_{i+1}_{layer.name}'

    # Freeze every submodel except for the 'active_index' one(if 'active_index' is set)
    # if active_index is not None:
    #     for i, sub_model in enumerate(sub_models):
    #         if i != active_index:
    #             for layer in sub_model.layers:
    #                 layer.trainable = False
    #         else:
    #             for layer in sub_model.layers:
    #                 layer.trainable = True
        
    # Create the ensemble model
    inputs = [sub_model.input for sub_model in sub_models]
    outputs = [sub_model.output for sub_model in sub_models]
    concat = tf.keras.layers.Concatenate()(outputs)
    ensemble_output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)
    ensemble_output = tf.keras.layers.Flatten()(ensemble_output)
    ensemble_model = tf.keras.Model(inputs=inputs, outputs=ensemble_output)

    # Compile the ensemble model
    ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(1e-1), loss='binary_crossentropy', metrics=['acc'])

    return ensemble_model

In [31]:
def freeze_unfreeze_submodels(ensemble_model, active_index):
    for i in range(len(ensemble_model.layers)):
        # print(i, ensemble_model.layers[i].name)
        if ensemble_model.layers[i].name.startswith(f"ensemble_{active_index}"):
            ensemble_model.layers[i].trainable = True
        elif ensemble_model.layers[i].name.startswith(f"ensemble_"):
            ensemble_model.layers[i].trainable = False
    
    ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(1e-1), loss='binary_crossentropy', metrics=['acc'])
    
    return ensemble_model

In [15]:
import sklearn.metrics as metrics
def evaluate_ensemble(model, X, y):
    '''Evaluates the ensemble model on the test set.'''
    
    y_preds = []
    for data in X:
        replicate_data = [tf.convert_to_tensor(np.array([data])) for _ in range(len(model.input))]
        y_pred = model(replicate_data)[0][0]
        y_pred = 1 if np.array(y_pred) > 0.5 else 0
        y_preds.append(y_pred)
    
    return metrics.accuracy_score(y, y_preds)

### Ensemble Experiment Functions


In [16]:
def elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates elman sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = create_elman_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(elman_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(elman_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(elman_submodels))]
    X_test_ensemble = [X_test for i in range(len(elman_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/elman_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    test_accuracy = evaluate_ensemble(ensemble_model, X_test, y_test)

    tf.keras.utils.plot_model(ensemble_model, to_file='elman_only_ensemble-{}.png'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), show_shapes=True)
    
    return ensemble_model, test_accuracy

In [17]:
def jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50]):
    '''Creates jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    jordan_submodels = create_jordan_submodels(hidden_count_list)

    # Create the ensemble model
    ensemble_model = create_ensemble_model(jordan_submodels)

    # Copy X_train and X_validation for all of the sub-models
    X_train_ensemble = [X_train for i in range(len(jordan_submodels))]
    X_validation_ensemble = [X_validation for i in range(len(jordan_submodels))]
    X_test_ensemble = [X_test for i in range(len(jordan_submodels))]
    
    # Train the ensemble model
    ensemble_model.fit(
        X_train_ensemble,
        y_train,
        batch_size=1,
        epochs=30,
        validation_data=(X_validation_ensemble, y_validation),
        callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/jordan_only_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    test_accuracy = evaluate_ensemble(ensemble_model, X_test, y_test)

    tf.keras.utils.plot_model(ensemble_model, to_file='jordan_only_ensemble-{}.png'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), show_shapes=True)

    return ensemble_model, test_accuracy

In [32]:
def endgame_ensemble_experiment(elman_hidden_count_list=[10, 25, 50], jordan_hidden_count_list=[10, 25, 50]):
    '''Creates elman and jordan sub-models, makes an ensemble out of them,
    trains the ensemble, reports test accuracy and returns the model.'''

    # Create the sub-models
    elman_submodels = []
    if len(elman_hidden_count_list)>0:
        elman_submodels = create_elman_submodels(elman_hidden_count_list)

    jordan_submodels = []
    if len(jordan_hidden_count_list)>0:
        jordan_submodels = create_jordan_submodels(jordan_hidden_count_list)

    # Total count of sub-models
    total_submodels = len(elman_submodels) + len(jordan_submodels)

    # Split X_train for each sub-model
    X_train_list = []
    y_train_list = []
    X_train_fixed = X_train[:-(X_train.shape[0]%total_submodels)]
    y_train_fixed = y_train[:-(X_train.shape[0]%total_submodels)]
    for i in range(total_submodels):
        X_train_current = X_train_fixed[int(i*X_train_fixed.shape[0]/total_submodels):int((i+1)*X_train_fixed.shape[0]/total_submodels)]
        y_train_current = y_train_fixed[int(i*X_train_fixed.shape[0]/total_submodels):int((i+1)*X_train_fixed.shape[0]/total_submodels)]
        X_train_list.append(X_train_current)
        y_train_list.append(y_train_current)

    # Choose X_train and copy X_validation for all of the sub-models
    X_train_ensemble = [X_train_list[i] for i in range(total_submodels)]
    y_train_ensemble = [y_train_list[i] for i in range(total_submodels)]
    X_validation_ensemble = [X_validation for i in range(total_submodels)]
    X_test_ensemble = [X_test for i in range(total_submodels)]
    
    ensemble_model = create_ensemble_model(elman_submodels + jordan_submodels)

    for i in range(total_submodels):
        # Freeze unfreeze
        ensemble_model = freeze_unfreeze_submodels(ensemble_model, active_index=i)

        # Train the ensemble model
        ensemble_model.fit(
            X_train_ensemble,
            y_train_ensemble[i],
            batch_size=1,
            epochs=100,
            validation_data=(X_validation_ensemble, y_validation),
            callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/endgame_ensemble-{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])

    # Report test accuracy
    test_accuracy = evaluate_ensemble(ensemble_model, X_test, y_test)

    tf.keras.utils.plot_model(ensemble_model, to_file='mixed_ensemble-{}.png'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), show_shapes=True)

    return ensemble_model, test_accuracy

## Main

### Loading the data

In [19]:
X, y = load_data()
print("X.shape:", X.shape)
print("X.dtype:", X.dtype)
print("y.shape:", y.shape)
print("y.dtype:", y.dtype)

X.shape: (463, 15, 6)
X.dtype: float32
y.shape: (463,)
y.dtype: float32


### Train, Test, validation split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.125, random_state=1)

In [21]:
print("X_train.shape:", X_train.shape)
print("X_validation.shape:", X_validation.shape)
print("X_test.shape:", X_test.shape)

X_train.shape: (323, 15, 6)
X_validation.shape: (47, 15, 6)
X_test.shape: (93, 15, 6)


### Normalize Dataset?

In [22]:
# Normalize the data

### Experiments

#### Elman

In [35]:
# Experiment with different hidden layer counts
elman_10, elman_10_score = train_elman(10)
elman_25, elman_25_score = train_elman(25)
elman_50, elman_50_score = train_elman(50)
elman_75, elman_75_score = train_elman(75)
elman_100, elman_100_score = train_elman(100)
elman_200, elman_200_score = train_elman(200)

Epoch 1/30
323/323 [==============================] - 3s 5ms/step - loss: 0.6412 - acc: 0.6873 - val_loss: 0.5712 - val_acc: 0.7872
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.5289 - acc: 0.7709 - val_loss: 0.4893 - val_acc: 0.7447
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.4593 - acc: 0.7957 - val_loss: 0.4253 - val_acc: 0.8298
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.4107 - acc: 0.8328 - val_loss: 0.4101 - val_acc: 0.7872
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3755 - acc: 0.8266 - val_loss: 0.4363 - val_acc: 0.7872
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3706 - acc: 0.7957 - val_loss: 0.3543 - val_acc: 0.8723
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3309 - acc: 0.8483 - val_loss: 0.3379 - val_acc: 0.8298
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [36]:
print("elman_10_score:", elman_10_score)
print("elman_25_score:", elman_25_score)
print("elman_50_score:", elman_50_score)
print("elman_75_score:", elman_75_score)
print("elman_100_score:", elman_100_score)
print("elman_200_score:", elman_200_score)

elman_10_score: 0.8602150537634409
elman_25_score: 0.8817204301075269
elman_50_score: 0.8924731182795699
elman_75_score: 0.946236559139785
elman_100_score: 0.9032258064516129
elman_200_score: 0.8924731182795699


#### Jordan

In [39]:
jordan_10, jordan_10_score = train_jordan(10)
jordan_25, jordan_25_score = train_jordan(25)
jordan_50, jordan_50_score = train_jordan(50)
jordan_75, jordan_75_score = train_jordan(75)
jordan_100, jordan_100_score = train_jordan(100)

Epoch 1/30
323/323 [==============================] - 3s 4ms/step - loss: 0.6359 - acc: 0.6873 - val_loss: 0.5098 - val_acc: 0.7660
Epoch 2/30
323/323 [==============================] - 1s 2ms/step - loss: 0.4554 - acc: 0.7957 - val_loss: 0.4404 - val_acc: 0.7447
Epoch 3/30
323/323 [==============================] - 1s 2ms/step - loss: 0.4087 - acc: 0.7895 - val_loss: 0.3529 - val_acc: 0.8085
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3640 - acc: 0.8173 - val_loss: 0.3396 - val_acc: 0.8298
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3492 - acc: 0.8266 - val_loss: 0.3478 - val_acc: 0.7872
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3305 - acc: 0.8111 - val_loss: 0.3317 - val_acc: 0.7872
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3139 - acc: 0.8452 - val_loss: 0.3338 - val_acc: 0.8085
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [40]:
print("jordan_10_score:", jordan_10_score)
print("jordan_25_score:", jordan_25_score)
print("jordan_50_score:", jordan_50_score)
print("jordan_75_score:", jordan_75_score)
print("jordan_100_score:", jordan_100_score)

jordan_10_score: 0.8602150537634409
jordan_25_score: 0.8817204301075269
jordan_50_score: 0.8924731182795699
jordan_75_score: 0.8924731182795699
jordan_100_score: 0.8064516129032258


### Ensemble

#### Elman Only 

In [41]:
elman_only_ensemble_3, elman_only_ensemble_3_score = elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50,])
elman_only_ensemble_4, elman_only_ensemble_4_score = elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50, 75])
elman_only_ensemble_5, elman_only_ensemble_5_score = elman_only_ensemble_experiment(hidden_count_list=[10, 25, 50, 75, 100])

Epoch 1/30
323/323 [==============================] - 5s 6ms/step - loss: 0.5571 - acc: 0.7074 - val_loss: 0.4930 - val_acc: 0.8298
Epoch 2/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4645 - acc: 0.7848 - val_loss: 0.4293 - val_acc: 0.8298
Epoch 3/30
323/323 [==============================] - 1s 3ms/step - loss: 0.4252 - acc: 0.8080 - val_loss: 0.4065 - val_acc: 0.8298
Epoch 4/30
323/323 [==============================] - 1s 2ms/step - loss: 0.4046 - acc: 0.8158 - val_loss: 0.3885 - val_acc: 0.8085
Epoch 5/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3776 - acc: 0.8266 - val_loss: 0.3634 - val_acc: 0.8298
Epoch 6/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3693 - acc: 0.8344 - val_loss: 0.3621 - val_acc: 0.7979
Epoch 7/30
323/323 [==============================] - 1s 2ms/step - loss: 0.3521 - acc: 0.8421 - val_loss: 0.3530 - val_acc: 0.8085
Epoch 8/30
323/323 [==============================] - 1s 2ms/step - loss: 0.

In [42]:
print("elman_only_ensemble_3_score:", elman_only_ensemble_3_score)
print("elman_only_ensemble_4_score:", elman_only_ensemble_4_score)
print("elman_only_ensemble_5_score:", elman_only_ensemble_5_score)

elman_only_ensemble_3_score: 0.9139784946236559
elman_only_ensemble_4_score: 0.8817204301075269
elman_only_ensemble_5_score: 0.9032258064516129


#### Jordan Only

In [43]:
jordan_only_ensemble_3, jordan_only_ensemble_3_score = jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50])
jordan_only_ensemble_4, jordan_only_ensemble_4_score = jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50, 75])
jordan_only_ensemble_5, jordan_only_ensemble_5_score = jordan_only_ensemble_experiment(hidden_count_list=[10, 25, 50, 75, 100])

Epoch 1/30
323/323 [==============================] - 8s 10ms/step - loss: 0.5934 - acc: 0.7136 - val_loss: 0.5044 - val_acc: 0.7872
Epoch 2/30
323/323 [==============================] - 1s 4ms/step - loss: 0.4690 - acc: 0.7926 - val_loss: 0.4432 - val_acc: 0.7660
Epoch 3/30
323/323 [==============================] - 1s 4ms/step - loss: 0.4411 - acc: 0.7523 - val_loss: 0.4133 - val_acc: 0.7660
Epoch 4/30
323/323 [==============================] - 1s 4ms/step - loss: 0.4131 - acc: 0.7337 - val_loss: 0.3993 - val_acc: 0.7872
Epoch 5/30
323/323 [==============================] - 1s 4ms/step - loss: 0.3873 - acc: 0.7957 - val_loss: 0.4084 - val_acc: 0.7021
Epoch 6/30
323/323 [==============================] - 1s 4ms/step - loss: 0.3719 - acc: 0.8251 - val_loss: 0.4085 - val_acc: 0.7660
Epoch 7/30
323/323 [==============================] - 1s 4ms/step - loss: 0.3598 - acc: 0.8390 - val_loss: 0.3903 - val_acc: 0.7872
Epoch 8/30
323/323 [==============================] - 1s 3ms/step - loss: 0

In [44]:
print("jordan_only_ensemble_3_score:", jordan_only_ensemble_3_score)
print("jordan_only_ensemble_4_score:", jordan_only_ensemble_4_score)
print("jordan_only_ensemble_5_score:", jordan_only_ensemble_5_score)

jordan_only_ensemble_3_score: 0.8709677419354839
jordan_only_ensemble_4_score: 0.9139784946236559
jordan_only_ensemble_5_score: 0.8602150537634409


#### Mixed

In [36]:
mixed_ensemble, mixed_ensemble_score = endgame_ensemble_experiment(
    elman_hidden_count_list=[75],
    jordan_hidden_count_list=[75])

Epoch 1/100
161/161 [==============================] - 4s 10ms/step - loss: 0.6443 - acc: 0.7019 - val_loss: 0.5745 - val_acc: 0.7447
Epoch 2/100
161/161 [==============================] - 1s 3ms/step - loss: 0.6261 - acc: 0.7019 - val_loss: 0.5708 - val_acc: 0.7447
Epoch 3/100
161/161 [==============================] - 1s 4ms/step - loss: 0.6364 - acc: 0.7019 - val_loss: 0.6185 - val_acc: 0.7447
Epoch 4/100
161/161 [==============================] - 1s 3ms/step - loss: 0.6369 - acc: 0.7019 - val_loss: 0.6689 - val_acc: 0.7447
Epoch 5/100
161/161 [==============================] - 1s 3ms/step - loss: 0.6485 - acc: 0.6708 - val_loss: 0.6099 - val_acc: 0.7447
Epoch 6/100
161/161 [==============================] - 1s 3ms/step - loss: 0.6316 - acc: 0.7019 - val_loss: 0.5919 - val_acc: 0.7447
Epoch 7/100
161/161 [==============================] - 1s 3ms/step - loss: 0.6265 - acc: 0.7019 - val_loss: 0.5790 - val_acc: 0.7447
Epoch 8/100
161/161 [==============================] - 1s 4ms/step -

In [38]:
evaluate_ensemble(mixed_ensemble, X_test, y_test)

0.6881720430107527

## Tensorboard

In [54]:
# load tensorboard extension
%load_ext tensorboard

# start tensorboard in the 'logs' directory
%tensorboard --logdir=logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11948), started 3 days, 7:55:32 ago. (Use '!kill 11948' to kill it.)

In [57]:
# start tensorboard in the 'logs' directory
%tensorboard --logdir=logs/

Reusing TensorBoard on port 6006 (pid 11948), started 3 days, 7:57:20 ago. (Use '!kill 11948' to kill it.)